# Using Uproot effectively

### If you're joining this tutorial live:

   1. Open this notebook in Binder or on your own computer (see [GitHub README](https://github.com/jpivarski-talks/2021-06-14-uproot-awkward-columnar-hats#readme)). If you have any troubles with installation or uninstalled libraries, switch to Binder.
   2. Open the Sli.do link that I'm sharing on the screen. This allows you to ask questions at any time, anonymously if you want.
   3. Evaluate the cells in the notebook along with me. I will frequently take detours, changing the input cells in real time. **Try out variations yourself, and ask questions at any time.**

Consider this a tour and I'm your tour guide. The notebook is a planned route to get things started, but your questions and wayfaring are more important.

### If you're reading this tutorial online or watching a video:

   * There's a directory of evaluated notebooks [in GitHub](https://github.com/jpivarski-talks/2021-06-14-uproot-awkward-columnar-hats), so you don't need to use Binder to see the cell output, but it's "canned," it doesn't include the experiments we tried on the day of the tutorial.
   * There are other tutorials on this subject. A very similar tutorial to this one will be presented at [PyHEP 2021](https://indico.cern.ch/event/1019958/) (July 5‒9, 2021). If that's in the future, join us! If that's in the past, the video is on YouTube.

## Python and Pythonic analysis tools (NumPy, Matplotlib, Pandas, etc.) are mainstream

This is an analysis of GitHub repos created by CMS physicists (i.e. "everyone who forked cms-sw/cmssw").

GitHub labels these repos as C/C++, Python, or Jupyter: the Python and Jupyter categories are now the most common.

<img src="img/lhlhc-github-languages.svg" style="width: 800px">

Furthermore, if we search for strings inside these repos, words like "`numpy`" are found in more repos than words like "`TFile`" (proxy for ROOT).

`"uproot"` is also fairly common, but not as much as the likes of NumPy and ROOT.

<img src="img/lhlhc-github-overlay-lin.svg" style="width: 800px">

We also asked questions about this at last year's PyHEP workshop (408 respondents, about 90 in CMS).

You use Python about equally with C++, and primarily for analysis (not just machine learning).

<img src="img/pyhep2020-survey-5.svg" style="width: 80%">

## Some general things about Python

   1. Python is fun and easy.
   2. Python is slow.

John Conway's game of life: [source](http://www.ericweisstein.com/encyclopedias/life/Puffer.html).

<img src="img/game-of-life-puffer.gif" style="width: 813px">

In [ ]:
WIDTH = 128
HEIGHT = 32

def new_world():
    world = [[0 for i in range(WIDTH)] for j in range(HEIGHT)]

    for x, y in [
        ( 4, 125), ( 3, 124), ( 3, 123), ( 3, 122), ( 3, 121), ( 3, 120), ( 3, 119), ( 4, 119), ( 5, 119), ( 6, 120),
        (10, 121), (11, 120), (12, 119), (12, 120), (13, 120), (13, 121), (14, 121),
        (20, 121), (19, 120), (18, 120), (18, 119), (17, 121), (17, 120), (16, 121),
        (26, 125), (27, 124), (27, 123), (27, 122), (27, 121), (27, 120), (27, 119), (26, 119), (25, 119), (24, 120)
    ]:
        world[x][y] = 1

    return world

world = new_world()

In [ ]:
def show(world):
    for row in world:
        stars = "".join("*" if cell else " " for cell in row)
        print("|" + stars + "|")

show(world)

In [ ]:
def step_python(world):
    outworld = []
    for i, row in enumerate(world):
        outrow = []
        for j, cell in enumerate(row):
            # count the number of living neighbors
            num_neighbors = 0
            for di in -1, 0, 1:
                for dj in -1, 0, 1:
                    if (di, dj) != (0, 0):
                        if world[(i + di) % HEIGHT][(j + dj) % WIDTH]:
                            num_neighbors += 1

            # use that information to decide if the next value of this cell is 0 or 1
            if cell and 1 < num_neighbors < 4:
                outrow.append(1)
            elif not cell and num_neighbors == 3:
                outrow.append(1)
            else:
                outrow.append(0)

        outworld.append(outrow)
    return outworld

In [ ]:
world = step_python(world)
show(world)

In [ ]:
%%timeit

step_python(world)

In [ ]:
%%writefile game-of-life.c

#include <stdint.h>

const int32_t WIDTH = 128;
const int32_t HEIGHT = 32;

void step_c(int8_t* inarray, int8_t* outarray) {
    for (int32_t i = 0; i < HEIGHT; i++) {
        for (int32_t j = 0; j < WIDTH; j++) {
            // count the number of living neighbors
            int32_t num_neighbors = 0;
            for (int32_t di = -1; di <= 1; di++) {
                for (int32_t dj = -1; dj <= 1; dj++) {
                    if (!(di == 0 && dj == 0)) {
                        if (inarray[((i + di + HEIGHT) % HEIGHT) * WIDTH + ((j + dj + WIDTH) % WIDTH)]) {
                            num_neighbors++;
                        }
                    }
                }
            }

            // use that information to decide if the next value of this cell is 0 or 1
            int8_t cell = inarray[i * WIDTH + j];
            if (cell && 1 < num_neighbors && num_neighbors < 4) {
                outarray[i * WIDTH + j] = 1;
            }
            else if (!cell && num_neighbors == 3) {
                outarray[i * WIDTH + j] = 1;
            }
            else {
                outarray[i * WIDTH + j] = 0;
            }
        }
    }

    // copy the outarray buffer to the inarray buffer (so the above can be repeated)
    for (int32_t i = 0; i < HEIGHT; i++) {
        for (int32_t j = 0; j < WIDTH; j++) {
            inarray[i * WIDTH + j] = outarray[i * WIDTH + j];
        }
    }
}

In [ ]:
!gcc -std=c99 game-of-life.c -O3 -shared -o game-of-life.so

In [ ]:
import ctypes

ArrayType = ctypes.c_int8 * WIDTH * HEIGHT

step_c = ctypes.cdll.LoadLibrary("./game-of-life.so").step_c
step_c.argtypes = [ArrayType, ArrayType]
step_c.restype = None

In [ ]:
inarray = ArrayType()
outarray = ArrayType()

for i, row in enumerate(new_world()):
    for j, cell in enumerate(row):
        inarray[i][j] = cell

In [ ]:
step_c(inarray, outarray, 1)
show(outarray)

In [ ]:
%%timeit

step_c(inarray, outarray)

In [ ]:
import numpy as np

def step_numpy(world):
    num_neighbors = np.zeros(world.shape, dtype=int)                        # initialize neighbors count
    num_neighbors += np.roll(np.roll(world,  1, axis=0),  1, axis=1)        # add southwest
    num_neighbors += np.roll(np.roll(world,  1, axis=0),  0, axis=1)        # add south
    num_neighbors += np.roll(np.roll(world,  1, axis=0), -1, axis=1)        # add southeast
    num_neighbors += np.roll(np.roll(world,  0, axis=0),  1, axis=1)        # add west
    num_neighbors += np.roll(np.roll(world,  0, axis=0), -1, axis=1)        # add east
    num_neighbors += np.roll(np.roll(world, -1, axis=0),  1, axis=1)        # add northwest
    num_neighbors += np.roll(np.roll(world, -1, axis=0),  0, axis=1)        # add north
    num_neighbors += np.roll(np.roll(world, -1, axis=0), -1, axis=1)        # add northeast

    survivors = ((world == 1) & (num_neighbors > 1) & (num_neighbors < 4))  # old cells that survive
    births    = ((world == 0) & (num_neighbors == 3))                       # new cells that are born

    return (births | survivors).astype(world.dtype)                         # union as booleans

world = np.array(new_world())

In [ ]:
world = step_numpy(world)
show(world)

In [ ]:
%%timeit

step_numpy(world)

In [ ]:
import scipy.signal

num_neighbors_convolver = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]])

def step_scipy(world):
    num_neighbors = scipy.signal.convolve2d(world, num_neighbors_convolver, mode="same", boundary="wrap")
    
    survivors = ((world == 1) & (num_neighbors > 1) & (num_neighbors < 4))  # old cells that survive
    births    = ((world == 0) & (num_neighbors == 3))                       # new cells that are born

    return (births | survivors).astype(world.dtype)                         # union as booleans

world = np.array(new_world())

In [ ]:
world = step_scipy(world)
show(world)

In [ ]:
%%timeit

step_scipy(world)

In [ ]:
import numba as nb

@nb.jit
def step_numba(world):
    outworld = np.empty_like(world)
    for i, row in enumerate(world):
        for j, cell in enumerate(row):
            # count the number of living neighbors
            num_neighbors = 0
            for di in -1, 0, 1:
                for dj in -1, 0, 1:
                    if (di, dj) != (0, 0):
                        if world[(i + di) % HEIGHT][(j + dj) % WIDTH]:
                            num_neighbors += 1

            # use that information to decide if the next value of this cell is 0 or 1
            if cell and 1 < num_neighbors < 4:
                outworld[i, j] = 1
            elif not cell and num_neighbors == 3:
                outworld[i, j] = 1
            else:
                outworld[i, j] = 0

    return outworld

world = np.array(new_world())

In [ ]:
world = step_numba(world)
show(world)

In [ ]:
%%timeit

step_numba(world)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

world = np.array(new_world())

fig = plt.figure(figsize=(8.13, 2.97), dpi=125)
plt.imshow(world)

In [ ]:
matplotlib.rc("animation", html="jshtml")
fig = plt.figure(figsize=(8.13, 2.97), dpi=125)

plt.figure(1)
graphic = plt.imshow(world)
plt.close(1)

def update(i):
    global world, graphic
    world = step_numba(world)
    graphic.set_array(world)
    return [graphic]

matplotlib.animation.FuncAnimation(fig, update, frames=250, interval=50, blit=True)

## What a ~complete analysis looks like in Uproot/Awkward Array

Instead of starting with small steps, let's look at where this is going, what a sample analysis looks like with these tools.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import awkward as ak

import uproot
import hist

In [ ]:
upfile = uproot.open("root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root")
uptree = upfile["Events"]
uptree.show()

The general strategy is to get arrays in one function call (usually slow, has to read) and use them interactively afterward.

In [ ]:
muons = uptree.arrays(["Muon_pt", "Muon_eta", "Muon_phi", "Muon_charge"], cut="nMuon >= 2", how="zip", entry_stop=100000)

We've already applied an `nMuon >= 2` cut, but we can define additional cuts.

In [ ]:
os_cut = muons[:, "Muon", "charge", 0] != muons[:, "Muon", "charge", 1]
os_cut

Slicing (to be described in more detail later) can filter and reduce the structure of an array.

In [ ]:
mu1 = muons[os_cut, 0, "Muon"]
mu2 = muons[os_cut, 1, "Muon"]
mu1, mu2

Make a histogram and fill it with a calculation from the array. The mini-plot is just the way this histogram type is visualized in Jupyter.

In [ ]:
h1 = hist.Hist.new.Reg(120, 0, 120, name="mass").Double()

In [ ]:
h1.fill(np.sqrt(2*mu1.pt*mu2.pt*(np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))))

Plot it using Matplotlib (for logscale).

In [ ]:
h1.plot()
plt.yscale("log")

## What a the same analysis looks like in PyROOT

In [ ]:
import ROOT
c1 = ROOT.TCanvas()

In [ ]:
rootfile = ROOT.TFile.Open("root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root")
roottree = rootfile.Get("Events")

ROOT analyses (before RDataFrame; see below) are based on an event loop. Reading and calculations are done in the loop.

In [ ]:
h2 = ROOT.TH1D("h2", "mass", 120, 0, 120)

for index, event in enumerate(roottree):
    # Analyzing a subsample means breaking out of the loop early.
    if index == 100000:
        break
    # Applying cuts means if-statements.
    if event.nMuon >= 2 and event.Muon_charge[0] != event.Muon_charge[1]:
        mu1_pt = event.Muon_pt[0]
        mu2_pt = event.Muon_pt[1]
        mu1_eta = event.Muon_eta[0]
        mu2_eta = event.Muon_eta[1]
        mu1_phi = event.Muon_phi[0]
        mu2_phi = event.Muon_phi[1]
        h2.Fill(np.sqrt(2*mu1_pt*mu2_pt*(np.cosh(mu1_eta - mu2_eta) - np.cos(mu1_phi - mu2_phi))))

In [ ]:
h2.Draw()
c1.SetLogy()
c1.Draw()

## What a the same analysis looks like in old C++

By "old C++," I mean `TTree::GetEntry`. This is also a reading + calculating loop over events.

Use `ROOT.gInterpreter.Declare` to define a C++ function in Python that we can use through PyROOT.

In [ ]:
ROOT.gInterpreter.Declare('''
void compute(TH1D& h3, TTree& roottree) {
    UInt_t nMuon;
    float Muon_pt[50];
    float Muon_eta[50];
    float Muon_phi[50];
    int32_t Muon_charge[50];

    roottree.SetBranchStatus("*", 0);
    roottree.SetBranchStatus("nMuon", 1);
    roottree.SetBranchStatus("Muon_pt", 1);
    roottree.SetBranchStatus("Muon_eta", 1);
    roottree.SetBranchStatus("Muon_phi", 1);
    roottree.SetBranchStatus("Muon_charge", 1);

    roottree.SetBranchAddress("nMuon", &nMuon);
    roottree.SetBranchAddress("Muon_pt", Muon_pt);
    roottree.SetBranchAddress("Muon_eta", Muon_eta);
    roottree.SetBranchAddress("Muon_phi", Muon_phi);
    roottree.SetBranchAddress("Muon_charge", Muon_charge);
    
    for (int index = 0; index < 100000; index++) {
        roottree.GetEntry(index);
        if (nMuon >= 2 && Muon_charge[0] != Muon_charge[1]) {
            float mu1_pt = Muon_pt[0];
            float mu2_pt = Muon_pt[1];
            float mu1_eta = Muon_eta[0];
            float mu2_eta = Muon_eta[1];
            float mu1_phi = Muon_phi[0];
            float mu2_phi = Muon_phi[1];
            h3.Fill(sqrt(2*mu1_pt*mu2_pt*(cosh(mu1_eta - mu2_eta) - cos(mu1_phi - mu2_phi))));
        }
    }
}
''')

In [ ]:
rootfile = ROOT.TFile.Open("root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root")
roottree = rootfile.Get("Events")

h3 = ROOT.TH1D("h3", "mass", 120, 0, 120)

ROOT.compute(h3, roottree)

In [ ]:
h3.Draw()
c1.SetLogy()
c1.Draw()

## What a the same analysis looks like in modern RDataFrame

This case mixes Python (for organization) with C++ (for speed).

<img src="img/rdataframe-flow.svg" style="width: 800px">

In [ ]:
df = ROOT.RDataFrame("Events", "root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root")

# Each node is connected to the previous, in a chain (which can split and recombine).
df_limit = df.Range(100000)
df_2mu = df_limit.Filter("nMuon >= 2")
df_os = df_2mu.Filter("Muon_charge[0] != Muon_charge[1]")

# This node is a big C++ block.
df_mass = df_os.Define("Dimuon_mass", '''
float mu1_pt = Muon_pt[0];
float mu2_pt = Muon_pt[1];
float mu1_eta = Muon_eta[0];
float mu2_eta = Muon_eta[1];
float mu1_phi = Muon_phi[0];
float mu2_phi = Muon_phi[1];
return sqrt(2*mu1_pt*mu2_pt*(cosh(mu1_eta - mu2_eta) - cos(mu1_phi - mu2_phi)));
''')

# This one is an endpoint (action).
h4 = df_mass.Histo1D(("h3", "mass", 120, 0, 120), "Dimuon_mass")

The above just sets up the calculation (compiling the C++ strings). It runs when you evaluate `h4.Draw`.

In [ ]:
h4.Draw()   # <--- This is the line that computes everything.
c1.SetLogy()
c1.Draw()

For more on RDataFrame, see [this tutorial](https://cms-opendata-workshop.github.io/workshop-lesson-root/05-rdataframe/index.html).